![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yields a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [2]:
rental_info = pd.read_csv('rental_info.csv')
rental_info.head()

rental_date                return_date  amount  release_year  \
0  2005-05-25 02:54:33+00:00  2005-05-28 23:40:33+00:00    2.99        2005.0   
1  2005-06-15 23:19:16+00:00  2005-06-18 19:24:16+00:00    2.99        2005.0   
2  2005-07-10 04:27:45+00:00  2005-07-17 10:11:45+00:00    2.99        2005.0   
3  2005-07-31 12:06:41+00:00  2005-08-02 14:30:41+00:00    2.99        2005.0   
4  2005-08-19 12:30:04+00:00  2005-08-23 13:35:04+00:00    2.99        2005.0   

   rental_rate  length  replacement_cost                special_features  \
0         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
1         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
2         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
3         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
4         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   

   NC-17  PG  PG-13  R  amount_2  length_2  rental_rate_2  
0      0   0      0  1    8.9401   15876.0         8.9401  
1      0   0      0  1    8.9401   15876.0         8.9401  
2      0   0      0  1    8.9401   15876.0         8.9401  
3      0   0      0  1    8.9401   15876.0         8.9401  
4      0   0      0  1    8.9401   15876.0         8.9401

In [3]:
# Data preprocessing
columns = rental_info.columns
for column in columns:
    print(rental_info[column].unique())

['2005-05-25 02:54:33+00:00' '2005-06-15 23:19:16+00:00'
 '2005-07-10 04:27:45+00:00' ... '2005-08-20 10:35:30+00:00'
 '2005-07-31 13:10:20+00:00' '2005-08-18 06:33:55+00:00']
['2005-05-28 23:40:33+00:00' '2005-06-18 19:24:16+00:00'
 '2005-07-17 10:11:45+00:00' ... '2005-08-29 13:03:30+00:00'
 '2005-08-08 14:07:20+00:00' '2005-08-24 07:14:55+00:00']
[ 2.99  4.99  3.99  0.99  1.99  6.99  5.99  8.99  7.99  9.99 10.99 11.99]
[2005. 2009. 2006. 2004. 2008. 2007. 2010.]
[2.99 0.99 4.99]
[126. 156. 181.  84.  92. 123.  56.  62. 109. 119. 184. 158.  48. 121.
 166.  81. 136. 147.  58. 182. 141. 112. 148. 135. 114. 165. 100. 130.
 153. 105.  77. 167.  63. 177. 111. 149. 137. 102. 161.  46. 144. 139.
 155. 173.  73. 172. 127.  54. 150. 122. 108. 180.  90. 110. 113.  74.
  88.  97. 140.  61. 162.  99. 176. 116.  64. 133. 107.  86.  93.  91.
  76.  78.  51.  71.  65.  75.  59.  85.  55. 183. 143.  94. 118. 185.
 145. 170.  47. 178. 129. 163. 152.  87. 103.  68. 169.  57.  89. 174.
 106.  83.  53. 

In [4]:
# Find out the number of rental days of a DVD
rental_info['rental_date'] = pd.to_datetime(rental_info['rental_date'])
rental_info['return_date'] = pd.to_datetime(rental_info['return_date'])

# Convert to UTC time to account for possible time shifts
rental_info['rental_date'] = rental_info['rental_date'].dt.tz_convert('UTC')
rental_info['return_date'] = rental_info['return_date'].dt.tz_convert('UTC')

# Separation and formatting of the time and date of rental and return DVDs
rental_info['rental_date_only'] = rental_info['rental_date'].dt.date
rental_info['rental_time_only'] = rental_info['rental_date'].dt.time
rental_info['return_date_only'] = rental_info['return_date'].dt.date
rental_info['return_time_only'] = rental_info['return_date'].dt.time

# Find rental length in days
rental_info['rental_length_days'] = (rental_info['return_date_only'] - rental_info['rental_date_only']).dt.days
rental_info.head()

rental_date               return_date  amount  release_year  \
0 2005-05-25 02:54:33+00:00 2005-05-28 23:40:33+00:00    2.99        2005.0   
1 2005-06-15 23:19:16+00:00 2005-06-18 19:24:16+00:00    2.99        2005.0   
2 2005-07-10 04:27:45+00:00 2005-07-17 10:11:45+00:00    2.99        2005.0   
3 2005-07-31 12:06:41+00:00 2005-08-02 14:30:41+00:00    2.99        2005.0   
4 2005-08-19 12:30:04+00:00 2005-08-23 13:35:04+00:00    2.99        2005.0   

   rental_rate  length  replacement_cost                special_features  \
0         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
1         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
2         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
3         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
4         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   

   NC-17  PG  PG-13  R  amount_2  length_2  rental_rate_2 rental_date_only  \
0      0   0      0  1    8.9401   15876.0         8.9401       2005-05-25   
1      0   0      0  1    8.9401   15876.0         8.9401       2005-06-15   
2      0   0      0  1    8.9401   15876.0         8.9401       2005-07-10   
3      0   0      0  1    8.9401   15876.0         8.9401       2005-07-31   
4      0   0      0  1    8.9401   15876.0         8.9401       2005-08-19   

  rental_time_only return_date_only return_time_only  rental_length_days  
0         02:54:33       2005-05-28         23:40:33                   3  
1         23:19:16       2005-06-18         19:24:16                   3  
2         04:27:45       2005-07-17         10:11:45                   7  
3         12:06:41       2005-08-02         14:30:41                   2  
4         12:30:04       2005-08-23         13:35:04                   4

In [5]:
# Further data preprocessing for special_features
rental_info['deleted_scenes'] = np.where(rental_info['special_features'].str.contains('Deleted Scenes'), 1, 0)
rental_info['behind_the_scenes'] = np.where(rental_info['special_features'].str.contains('Behind the Scenes'), 1,0)
rental_info.head()

rental_date               return_date  amount  release_year  \
0 2005-05-25 02:54:33+00:00 2005-05-28 23:40:33+00:00    2.99        2005.0   
1 2005-06-15 23:19:16+00:00 2005-06-18 19:24:16+00:00    2.99        2005.0   
2 2005-07-10 04:27:45+00:00 2005-07-17 10:11:45+00:00    2.99        2005.0   
3 2005-07-31 12:06:41+00:00 2005-08-02 14:30:41+00:00    2.99        2005.0   
4 2005-08-19 12:30:04+00:00 2005-08-23 13:35:04+00:00    2.99        2005.0   

   rental_rate  length  replacement_cost                special_features  \
0         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
1         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
2         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
3         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
4         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   

   NC-17  PG  ...  amount_2  length_2  rental_rate_2  rental_date_only  \
0      0   0  ...    8.9401   15876.0         8.9401        2005-05-25   
1      0   0  ...    8.9401   15876.0         8.9401        2005-06-15   
2      0   0  ...    8.9401   15876.0         8.9401        2005-07-10   
3      0   0  ...    8.9401   15876.0         8.9401        2005-07-31   
4      0   0  ...    8.9401   15876.0         8.9401        2005-08-19   

   rental_time_only return_date_only return_time_only rental_length_days  \
0          02:54:33       2005-05-28         23:40:33                  3   
1          23:19:16       2005-06-18         19:24:16                  3   
2          04:27:45       2005-07-17         10:11:45                  7   
3          12:06:41       2005-08-02         14:30:41                  2   
4          12:30:04       2005-08-23         13:35:04                  4   

  deleted_scenes  behind_the_scenes  
0              0                  1  
1              0                  1  
2              0                  1  
3              0                  1  
4              0                  1  

[5 rows x 22 columns]

In [6]:
# Train-test split
cols = ['amount', 'release_year', 'rental_rate', 'length', 'NC-17', 'PG', 'PG-13', 'R', 'deleted_scenes', 'behind_the_scenes', 'replacement_cost', 'amount_2','length_2', 'rental_rate_2']
X = rental_info[cols].values
y = rental_info['rental_length_days'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 9)

In [7]:
# ML Model 1: Unscaled linear regression model
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_test)
r2_unscaled = linreg.score(X_test, y_test)

# Calculate MSE for the test and its mean values, baseline model 
y_baseline_pred = np.full_like(y_test, np.mean(y_test))
baseline_mse_unscaled = mean_squared_error(y_test, y_baseline_pred)

# Calculate MSE for our model MSE(baseline)>MSE(model) = better model's accuracy
mse_linreg_unscaled = mean_squared_error(y_test, y_pred, squared = False)

# Compare baseline to actual model
print(f'The baseline MSE performance is: {baseline_mse_unscaled}')
print(f'The MSE performance of the unscaled linear regression mode is: {mse_linreg_unscaled}')

The baseline MSE performance is: 6.90356129845572
The MSE performance of the unscaled linear regression mode is: 1.681131196458586


In [8]:
# ML Model 2: Scaled linear regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
y_train_scaled = scaler.fit_transform(y_train.reshape(-1,1))
y_test_scaled = scaler.transform(y_test.reshape(-1,1))

linreg.fit(X_train_scaled, y_train_scaled)
y_pred_scaled = linreg.predict(X_test_scaled)
r2_scaled = linreg.score(X_test_scaled, y_test_scaled)

# Calculate MSE for the test and its mean values, baseline model 
y_baseline_pred_scaled = np.full_like(y_test_scaled, np.mean(y_test_scaled))
baseline_mse_scaled = mean_squared_error(y_test_scaled, y_baseline_pred_scaled)

# Calculate MSE for our model MSE(baseline)>MSE(model) = better model's accuracy
mse_linreg_scaled = mean_squared_error(y_test_scaled, y_pred_scaled, squared = False)

# Compare baseline to actual model
print(f'The baseline MSE performance is: {baseline_mse_scaled}')
print(f'The MSE performance of the unscaled linear regression mode is: {mse_linreg_scaled}')

The baseline MSE performance is: 1.0199484821503462
The MSE performance of the unscaled linear regression mode is: 0.6462017915030145


In [9]:
# ML Model 3: Scaled linear regression with cross-validation
kf = KFold(n_splits = 6, shuffle = True, random_state = 9)
mse_scores = - cross_val_score(linreg, X_train_scaled, y_train_scaled, cv = kf,
                            scoring = 'neg_mean_squared_error')
mse_linreg_scaled_cross_val = np.mean(mse_scores)

print(f'The MSE performance of the scaled linear regression model with cross-validation is: {mse_linreg_scaled_cross_val}.')

The MSE performance of the scaled linear regression model with cross-validation is: 0.4086372251793618.


In [10]:
# ML Model 4: Scaled Lasso regression with cross-validation
param_grid_lasso = {'alpha': np.arange(0.000001,0.1,10),
              'fit_intercept': [True, False]}
              #'selection': ['cyclic', 'random']}
lasreg = Lasso()
lasso_cv = GridSearchCV(lasreg, param_grid_lasso, cv = kf, scoring='neg_mean_squared_error')
lasso_cv.fit(X_train_scaled, y_train_scaled)
mse_lasso = - lasso_cv.best_score_
params_lasso = lasso_cv.best_params_
print(f'The MSE performance of the scaled Lasso regression model with cross-validation is: {mse_lasso}')
print(f'The optimal Lasso model parameters are: {params_lasso}')

The MSE performance of the scaled Lasso regression model with cross-validation is: 0.40858079855186724
The optimal Lasso model parameters are: {'alpha': 1e-06, 'fit_intercept': False}


In [11]:
# ML Model 5: Scaled Ridge regression with cross-validation
param_grid_ridge = {'alpha': np.arange(0.000001,0.1,10),
              'fit_intercept': [True, False],
              'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs']}
ridgereg = Ridge()
ridge_cv = GridSearchCV(ridgereg, param_grid_ridge, cv = kf, scoring='neg_mean_squared_error')
ridge_cv.fit(X_train_scaled, y_train_scaled)
mse_ridge = - ridge_cv.best_score_
params_ridge = ridge_cv.best_params_
print(f'The MSE performance of the scaled Ridge regression model with cross-validation is: {mse_ridge}')
print(f'The optimal Ridge model parameters are: {params_ridge}')

The MSE performance of the scaled Ridge regression model with cross-validation is: 0.4085659275190656
The optimal Ridge model parameters are: {'alpha': 1e-06, 'fit_intercept': False, 'solver': 'saga'}


C:\Users\Dimitrios Gkiatas\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
12 fits failed out of a total of 96.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Dimitrios Gkiatas\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Dimitrios Gkiatas\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py", line 1011, in fit
    return super().fit(X, y, sample_weight=sample_weight)
  File "C:\Users\Dimitrios Gkiatas\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py", line 705, in fit
 

In [12]:
# Dictionary of different model performance
models_performance = {'Linear Regression':mse_linreg_scaled_cross_val,
                     'Lasso Regression': mse_lasso,
                     'Ridge Regression': mse_lasso}

best_model = min(models_performance, key = models_performance.get)
best_mse = models_performance[best_model]

print(f'The best performing model *{best_model}*, yields a MSE score of *{best_mse}*.')

The best performing model *Lasso Regression*, yields a MSE score of *0.40858079855186724*.
